# ERA5-Land Heat Metrics Map Explorer

Interactive map-based tool to explore heat wave patterns and temperature extremes using real ERA5-Land data.
Analyze trends and dynamics across countries with user-defined metrics, thresholds, and time periods.

In [1]:
# Import libraries
import ee
import geemap
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import ipywidgets as widgets
from IPython.display import display, clear_output
import warnings
warnings.filterwarnings('ignore')

# Initialize Google Earth Engine
try:
    ee.Initialize(project='tl-cities')
    print("✅ Google Earth Engine initialized")
except Exception as e:
    print(f"❌ GEE initialization failed: {e}")

print("🌡️ ERA5-Land Heat Metrics Explorer Ready")

✅ Google Earth Engine initialized
🌡️ ERA5-Land Heat Metrics Explorer Ready


## Load Administrative Boundaries and ERA5-Land Data

In [2]:
# Load geoBoundaries administrative boundaries
print("Loading geoBoundaries administrative boundaries...")
try:
    # Load geoBoundaries Level 0 (countries) and Level 3 (districts)
    gadm0 = ee.FeatureCollection("projects/sat-io/open-datasets/geoboundaries/CGAZ_ADM0")
    gadm3 = ee.FeatureCollection("projects/sat-io/open-datasets/geoboundaries/CGAZ_ADM2")  # ADM2 is typically districts
    print("✅ geoBoundaries loaded successfully")
except Exception as e:
    print(f"❌ geoBoundaries loading failed: {e}")
    print("Trying alternative geoBoundaries paths...")
    try:
        # Try alternative paths
        gadm0 = ee.FeatureCollection("projects/sat-io/open-datasets/geoboundaries/geoBoundariesCGAZ-ADM0")
        gadm3 = ee.FeatureCollection("projects/sat-io/open-datasets/geoboundaries/geoBoundariesCGAZ-ADM2")
        print("✅ Alternative geoBoundaries loaded successfully")
    except Exception as e2:
        print(f"❌ Alternative geoBoundaries loading failed: {e2}")
        print("Using simple country boundaries...")
        gadm0 = ee.FeatureCollection("USDOS/LSIB_SIMPLE/2017")
        gadm3 = None

# Load ERA5-Land dataset
print("Loading ERA5-Land dataset...")
try:
    era5_land = ee.ImageCollection("ECMWF/ERA5_LAND/DAILY_AGGR")
    print("✅ ERA5-Land dataset loaded successfully")
    print(f"   Available from: 1981-01-01 to present")
    print(f"   Spatial resolution: ~11km")
except Exception as e:
    print(f"❌ ERA5-Land loading failed: {e}")

# Get list of countries
print("Extracting country list...")
try:
    # Check what fields are available in the first feature
    sample_props = gadm0.limit(1).getInfo()['features'][0]['properties']
    print(f"Available fields: {list(sample_props.keys())}")
    
    # Try different field names for country extraction
    country_field = None
    for field in ['shapeName', 'shapeGroup', 'shapeISO', 'ADM0_NAME', 'NAME_EN', 'COUNTRY', 'country_na']:
        if field in sample_props:
            country_field = field
            break
    
    if country_field:
        countries = gadm0.distinct(country_field).aggregate_array(country_field).getInfo()
        countries = sorted([c for c in countries if c and str(c) != 'Unknown' and len(str(c)) > 1])
        print(f"✅ Loaded {len(countries)} countries using field '{country_field}'")
    else:
        raise Exception("No suitable country field found")
        
except Exception as e:
    print(f"❌ Country extraction failed: {e}")
    # Fallback to predefined list of major countries
    countries = ['Australia', 'Brazil', 'Canada', 'China', 'France', 'Germany', 'India', 'Italy', 
                'Japan', 'Mexico', 'Russia', 'South Africa', 'Spain', 'Turkey', 'United Kingdom', 
                'United States of America']
    country_field = 'shapeName'  # Default field name for geoBoundaries
    print(f"Using fallback country list with {len(countries)} countries")

Loading geoBoundaries administrative boundaries...
✅ geoBoundaries loaded successfully
Loading ERA5-Land dataset...
✅ ERA5-Land dataset loaded successfully
   Available from: 1981-01-01 to present
   Spatial resolution: ~11km
Extracting country list...
Available fields: ['shapeGroup', 'shapeID', 'shapeISO', 'shapeName', 'shapeType']
✅ Loaded 197 countries using field 'shapeName'


## Heat Metrics Calculation Functions

In [3]:
def calculate_temperature_percentiles(country_geom, baseline_start='1981-01-01', baseline_end='2010-12-31'):
    """
    Calculate temperature percentiles for baseline period (climate normals)
    """
    baseline_collection = era5_land.filterDate(baseline_start, baseline_end) \
                                   .filterBounds(country_geom) \
                                   .select(['temperature_2m'])
    
    # Convert from Kelvin to Celsius
    baseline_celsius = baseline_collection.map(lambda img: img.subtract(273.15))
    
    # Calculate percentiles
    p90 = baseline_celsius.reduce(ee.Reducer.percentile([90])).rename('temp_p90')
    p95 = baseline_celsius.reduce(ee.Reducer.percentile([95])).rename('temp_p95')
    p99 = baseline_celsius.reduce(ee.Reducer.percentile([99])).rename('temp_p99')
    
    return p90, p95, p99

def calculate_exceedance_days(country_geom, year, threshold_type='fixed', threshold_value=30, percentile_img=None):
    """
    Calculate number of days exceeding temperature threshold
    """
    year_collection = era5_land.filterDate(f'{year}-01-01', f'{year}-12-31') \
                               .filterBounds(country_geom) \
                               .select(['temperature_2m'])
    
    # Convert to Celsius
    year_celsius = year_collection.map(lambda img: img.subtract(273.15))
    
    if threshold_type == 'fixed':
        # Fixed temperature threshold
        exceedance = year_celsius.map(lambda img: img.gt(threshold_value))
    else:
        # Percentile-based threshold
        exceedance = year_celsius.map(lambda img: img.gt(percentile_img))
    
    # Count exceedance days
    exceedance_days = exceedance.sum().rename('exceedance_days')
    
    return exceedance_days

def calculate_ehf_heat_wave_days(country_geom, year, p95_baseline, ehf_threshold=0):
    """
    Calculate heat wave days using EHF methodology
    Returns count of days with different EHF conditions
    """
    year_collection = era5_land.filterDate(f'{year}-01-01', f'{year}-12-31') \
                               .filterBounds(country_geom) \
                               .select(['temperature_2m'])
    
    # Convert to Celsius
    year_celsius = year_collection.map(lambda img: img.subtract(273.15))
    
    # Convert to list for processing
    temp_list = year_celsius.toList(year_celsius.size())
    
    def calculate_daily_ehf(i):
        i = ee.Number(i)
        
        # Get 3-day average (current and previous 2 days)
        start_idx = ee.Number(ee.Algorithms.If(i.gte(2), i.subtract(2), 0))
        end_idx = i.add(1)
        
        three_day_imgs = temp_list.slice(start_idx, end_idx)
        three_day_mean = ee.ImageCollection(three_day_imgs).mean()
        
        # Get 30-day average (previous 30 days)
        start_30 = ee.Number(ee.Algorithms.If(i.gte(30), i.subtract(30), 0))
        thirty_day_imgs = temp_list.slice(start_30, i.add(1))
        thirty_day_mean = ee.ImageCollection(thirty_day_imgs).mean()
        
        # Calculate EHI components
        ehi_accl = three_day_mean.subtract(thirty_day_mean)  # Acclimatization component
        ehi_sig = three_day_mean.subtract(p95_baseline)      # Significance component
        
        # EHF = max(1, EHI_accl) × EHI_sig (when EHI_sig > 0)
        ehf = ehi_accl.max(1).multiply(ehi_sig).updateMask(ehi_sig.gt(0))
        
        return ehf.set('system:time_start', ee.Image(temp_list.get(i)).get('system:time_start'))
    
    # Calculate EHF for all days
    sequence = ee.List.sequence(0, year_celsius.size().subtract(1))
    ehf_collection = ee.ImageCollection(sequence.map(calculate_daily_ehf))
    
    # Return different day count metrics based on EHF thresholds
    if ehf_threshold == 0:
        # Heat wave days (any positive EHF)
        heat_wave_days = ehf_collection.map(lambda img: img.gt(0)).sum().rename('heat_wave_days')
        return heat_wave_days
    elif ehf_threshold == 5:
        # Moderate heat stress days (EHF > 5)
        moderate_days = ehf_collection.map(lambda img: img.gt(5)).sum().rename('moderate_heat_days')
        return moderate_days
    elif ehf_threshold == 15:
        # Severe heat stress days (EHF > 15)
        severe_days = ehf_collection.map(lambda img: img.gt(15)).sum().rename('severe_heat_days')
        return severe_days
    else:
        # Extreme heat stress days (EHF > 30)
        extreme_days = ehf_collection.map(lambda img: img.gt(30)).sum().rename('extreme_heat_days')
        return extreme_days

def calculate_temperature_threshold_days(country_geom, year, percentile_img, threshold_value):
    """
    Calculate days above a manual temperature threshold applied to percentile temperatures
    """
    year_collection = era5_land.filterDate(f'{year}-01-01', f'{year}-12-31') \
                               .filterBounds(country_geom) \
                               .select(['temperature_2m'])
    
    # Convert to Celsius
    year_celsius = year_collection.map(lambda img: img.subtract(273.15))
    
    # Create threshold: percentile + manual adjustment
    adjusted_threshold = percentile_img.add(threshold_value)
    
    # Count days exceeding adjusted threshold
    exceedance = year_celsius.map(lambda img: img.gt(adjusted_threshold))
    exceedance_days = exceedance.sum().rename('adjusted_threshold_days')
    
    return exceedance_days

def get_country_geometry(country_name):
    """
    Get country geometry from geoBoundaries dataset
    """
    global country_field
    
    # Use the detected country field from the loading step
    try:
        country = gadm0.filter(ee.Filter.eq(country_field, country_name))
        count = country.size().getInfo()
        if count > 0:
            return country.geometry()
        else:
            # Try partial match if exact match fails
            country = gadm0.filter(ee.Filter.stringContains(country_field, country_name))
            count = country.size().getInfo()
            if count > 0:
                return country.first().geometry()
    except Exception as e:
        print(f"Error getting country geometry: {e}")
    
    # Try alternative field names as fallback
    for field in ['shapeName', 'shapeGroup', 'shapeISO', 'ADM0_NAME', 'NAME_EN', 'COUNTRY', 'country_na']:
        try:
            country = gadm0.filter(ee.Filter.eq(field, country_name))
            if country.size().getInfo() > 0:
                return country.geometry()
        except:
            continue
    
    raise ValueError(f"Country '{country_name}' not found in boundary dataset")

def get_gadm3_districts(country_name):
    """
    Get geoBoundaries Level 2 districts for a country (ADM2 level)
    """
    if gadm3 is None:
        return None
    
    # Try different field names for country matching in ADM2 data
    for field in ['shapeGroup', 'shapeName', 'ADM0_NAME', 'NAME_0', 'COUNTRY']:
        try:
            districts = gadm3.filter(ee.Filter.eq(field, country_name))
            count = districts.size().getInfo()
            if count > 0:
                return districts
        except:
            continue
    
    # Try partial matches
    try:
        districts = gadm3.filter(ee.Filter.stringContains('shapeGroup', country_name))
        count = districts.size().getInfo()
        if count > 0:
            return districts
    except:
        pass
    
    return None

print("✅ Heat metrics calculation functions defined (all return day counts)")

✅ Heat metrics calculation functions defined (all return day counts)


## Visualization and Mapping Functions

In [ ]:
def update_map_display(m, image, metric_name, country_name, year, show_districts=False, districts=None):
    """
    Update map with new heat metric visualization
    """
    # Simplified layer clearing - just try basic approach
    try:
        # Try to get layer names and remove non-base layers
        if hasattr(m, '_layers'):
            # Remove layers by iterating through a copy of the layer list
            layer_names_to_remove = []
            for layer in m._layers:
                if hasattr(layer, 'name') and layer.name:
                    if 'satellite' not in layer.name.lower() and 'roadmap' not in layer.name.lower():
                        layer_names_to_remove.append(layer.name)
            
            for name in layer_names_to_remove:
                try:
                    m.remove_layer(name)
                except:
                    pass
    except Exception as e:
        # If layer clearing fails, just continue - new layers will be added on top
        print(f"Note: Could not clear layers, adding new layer on top")
    
    try:
        # Get country geometry for centering and stats
        country_geom = get_country_geometry(country_name)
        
        # Don't clip the image to country geometry - show the full regional data
        # This will ensure we see all available data in the region
        region_bounds = country_geom.bounds().buffer(100000)  # Add 100km buffer
        
        # Calculate statistics over the full country (not just center)
        stats = image.reduceRegion(
            reducer=ee.Reducer.minMax(),
            geometry=country_geom,
            scale=50000,  # Larger scale for better coverage
            maxPixels=1e7,
            bestEffort=True
        )
        
        stats_info = stats.getInfo()
        print(f"Statistics computed: {stats_info}")
        
        # Get the correct field names for min/max
        possible_names = [metric_name, 'exceedance_days', 'heat_wave_days', 'moderate_heat_days', 'severe_heat_days', 'extreme_heat_days', 'adjusted_threshold_days']
        min_val = 0
        max_val = 50  # Default reasonable max for day counts
        
        for name in possible_names:
            min_key = f'{name}_min'
            max_key = f'{name}_max'
            if min_key in stats_info and max_key in stats_info:
                min_val = stats_info[min_key] or 0
                max_val = stats_info[max_key] or 50
                print(f"Found stats for {name}: {min_val} - {max_val}")
                break
        
        # Ensure reasonable range
        if max_val <= min_val:
            min_val = 0
            max_val = 50
        
        # Create visualization parameters
        vis_params = create_heat_visualization_params(metric_name, image)
        vis_params['min'] = min_val
        vis_params['max'] = max_val
        
        # Create a unique layer name
        layer_name = f'{metric_name}_{country_name}_{year}'.replace(' ', '_')
        
        # Add the heat metric layer WITHOUT clipping to show full regional data
        print(f"Adding layer: {layer_name}")
        m.addLayer(
            image,  # Don't clip - show full data
            vis_params,
            layer_name
        )
        
        # Add country boundary outline for reference
        country_style = {
            'color': 'blue',
            'fillColor': '00000000',  # Transparent fill
            'width': 2,
            'opacity': 1.0
        }
        m.addLayer(country_geom, country_style, f'{country_name}_Boundary')
        
        # Add district boundaries if requested
        if show_districts and districts is not None:
            district_style = {
                'color': 'black',
                'fillColor': '00000000',
                'width': 1,
                'opacity': 0.7
            }
            district_layer_name = f'Districts_{country_name}'
            m.addLayer(districts, district_style, district_layer_name)
            print(f"Added district boundaries: {district_layer_name}")
        
        # Center and zoom to country
        m.centerObject(country_geom, 4)  # Zoom out a bit more to see full country
        print(f"Map centered on {country_name}")
        
        return True, f"✅ Map updated: {metric_name} for {country_name} ({year}) - Range: {min_val:.0f} to {max_val:.0f} days"
        
    except Exception as e:
        import traceback
        error_details = traceback.format_exc()
        return False, f"❌ Error updating map: {str(e)}\nDetails: {error_details}"

print("✅ Visualization functions updated to show full regional data coverage")

## Interactive User Interface

In [5]:
# Create interactive map
m = geemap.Map(center=[20, 0], zoom=2)

# Create control widgets
country_selector = widgets.Dropdown(
    options=['Select Country...'] + countries,
    value='Select Country...',
    description='Country:',
    style={'description_width': '100px'},
    layout=widgets.Layout(width='300px')
)

year_selector = widgets.Dropdown(
    options=list(range(1981, 2024)),
    value=2023,
    description='Year:',
    style={'description_width': '100px'},
    layout=widgets.Layout(width='200px')
)

baseline_selector = widgets.Dropdown(
    options=[('1981-2010', '1981-2010'), ('1961-1990', '1961-1990')],
    value='1981-2010',
    description='Baseline:',
    style={'description_width': '100px'},
    layout=widgets.Layout(width='200px')
)

metric_selector = widgets.Dropdown(
    options=[
        ('Days > Fixed Threshold', 'exceedance_fixed'),
        ('Days > 90th Percentile', 'exceedance_p90'),
        ('Days > 95th Percentile', 'exceedance_p95'),
        ('Days > 99th Percentile', 'exceedance_p99'),
        ('Heat Wave Days (EHF > 0)', 'heat_wave_basic'),
        ('Moderate Heat Stress Days (EHF > 5)', 'heat_wave_moderate'),
        ('Severe Heat Stress Days (EHF > 15)', 'heat_wave_severe'),
        ('Extreme Heat Stress Days (EHF > 30)', 'heat_wave_extreme'),
        ('Days > Percentile + Adjustment', 'adjusted_threshold')
    ],
    value='exceedance_p95',
    description='Metric:',
    style={'description_width': '100px'},
    layout=widgets.Layout(width='350px')
)

# Always visible threshold slider
threshold_slider = widgets.FloatSlider(
    value=30.0,
    min=20.0,
    max=50.0,
    step=1.0,
    description='Manual Threshold (°C):',
    style={'description_width': '130px'},
    layout=widgets.Layout(width='350px')
)

# Adjustment slider for percentile-based thresholds
adjustment_slider = widgets.FloatSlider(
    value=0.0,
    min=-10.0,
    max=10.0,
    step=0.5,
    description='Percentile Adjustment (°C):',
    style={'description_width': '130px'},
    layout=widgets.Layout(width='350px')
)

districts_toggle = widgets.Checkbox(
    value=False,
    description='Show District Boundaries',
    style={'description_width': '150px'}
)

update_button = widgets.Button(
    description='🔄 Update Map',
    button_style='primary',
    layout=widgets.Layout(width='200px', height='35px')
)

status_output = widgets.Output(layout=widgets.Layout(height='120px'))

# Global variables to store computed data
current_percentiles = {}
current_country = None
current_districts = None

# Make country_field available globally (set during data loading)
try:
    country_field = country_field  # This should be set from the loading step
except NameError:
    country_field = 'shapeName'  # Default fallback

print("✅ User interface widgets created with always-visible threshold controls")

✅ User interface widgets created with always-visible threshold controls


## Event Handlers and Main Analysis Function

In [6]:
def update_slider_visibility():
    """Show/hide sliders based on selected metric"""
    metric = metric_selector.value
    
    if metric == 'exceedance_fixed':
        threshold_slider.layout.visibility = 'visible'
        adjustment_slider.layout.visibility = 'hidden'
    elif metric == 'adjusted_threshold':
        threshold_slider.layout.visibility = 'hidden'
        adjustment_slider.layout.visibility = 'visible'
    else:
        threshold_slider.layout.visibility = 'hidden'
        adjustment_slider.layout.visibility = 'hidden'

def compute_heat_metric():
    """
    Main function to compute and display selected heat metric
    All metrics now return day counts
    """
    global current_percentiles, current_country, current_districts
    
    with status_output:
        clear_output(wait=True)
        
        country = country_selector.value
        year = year_selector.value
        baseline = baseline_selector.value
        metric = metric_selector.value
        threshold = threshold_slider.value
        adjustment = adjustment_slider.value
        show_districts = districts_toggle.value
        
        if country == 'Select Country...':
            print("⚠️ Please select a country")
            return
        
        print(f"🔄 Processing {metric} for {country} ({year})...")
        print(f"   Baseline period: {baseline}")
        
        try:
            # Get country geometry
            country_geom = get_country_geometry(country)
            
            # Load districts if needed
            if show_districts and (current_country != country or current_districts is None):
                print("   Loading district boundaries...")
                current_districts = get_gadm3_districts(country)
                current_country = country
            
            # Calculate baseline percentiles if needed
            baseline_key = f"{country}_{baseline}"
            if baseline_key not in current_percentiles:
                print("   Calculating baseline percentiles...")
                if baseline == '1981-2010':
                    p90, p95, p99 = calculate_temperature_percentiles(country_geom, '1981-01-01', '2010-12-31')
                else:  # 1961-1990
                    p90, p95, p99 = calculate_temperature_percentiles(country_geom, '1961-01-01', '1990-12-31')
                
                current_percentiles[baseline_key] = {
                    'p90': p90,
                    'p95': p95,
                    'p99': p99
                }
            
            percentiles = current_percentiles[baseline_key]
            
            # Calculate selected metric - all return day counts
            print(f"   Calculating {metric}...")
            
            if metric == 'exceedance_fixed':
                result_image = calculate_exceedance_days(country_geom, year, 'fixed', threshold)
                display_name = 'exceedance_days'
                
            elif metric == 'exceedance_p90':
                result_image = calculate_exceedance_days(country_geom, year, 'percentile', None, percentiles['p90'])
                display_name = 'exceedance_days'
                
            elif metric == 'exceedance_p95':
                result_image = calculate_exceedance_days(country_geom, year, 'percentile', None, percentiles['p95'])
                display_name = 'exceedance_days'
                
            elif metric == 'exceedance_p99':
                result_image = calculate_exceedance_days(country_geom, year, 'percentile', None, percentiles['p99'])
                display_name = 'exceedance_days'
                
            elif metric == 'heat_wave_basic':
                result_image = calculate_ehf_heat_wave_days(country_geom, year, percentiles['p95'], 0)
                display_name = 'heat_wave_days'
                
            elif metric == 'heat_wave_moderate':
                result_image = calculate_ehf_heat_wave_days(country_geom, year, percentiles['p95'], 5)
                display_name = 'moderate_heat_days'
                
            elif metric == 'heat_wave_severe':
                result_image = calculate_ehf_heat_wave_days(country_geom, year, percentiles['p95'], 15)
                display_name = 'severe_heat_days'
                
            elif metric == 'heat_wave_extreme':
                result_image = calculate_ehf_heat_wave_days(country_geom, year, percentiles['p95'], 30)
                display_name = 'extreme_heat_days'
                
            elif metric == 'adjusted_threshold':
                result_image = calculate_temperature_threshold_days(country_geom, year, percentiles['p95'], adjustment)
                display_name = 'adjusted_threshold_days'
            
            # Update map display
            print("   Updating map display...")
            success, message = update_map_display(
                m, result_image, display_name, country, year, 
                show_districts, current_districts
            )
            
            if success:
                print(message)
            else:
                print(message)
                
        except Exception as e:
            print(f"❌ Error: {str(e)}")
            import traceback
            print(f"   Details: {traceback.format_exc()}")

# Event handlers
def on_metric_change(change):
    update_slider_visibility()

def on_update_button_click(b):
    compute_heat_metric()

# Connect event handlers
metric_selector.observe(on_metric_change, names='value')
update_button.on_click(on_update_button_click)

# Initialize slider visibility
update_slider_visibility()

print("✅ Event handlers connected with updated metric calculations")

✅ Event handlers connected with updated metric calculations


## Display Interactive Interface

In [7]:
# Create control panel layout
controls_row1 = widgets.HBox([
    country_selector,
    year_selector,
    baseline_selector
])

controls_row2 = widgets.HBox([
    metric_selector
])

controls_row3 = widgets.HBox([
    threshold_slider,
    adjustment_slider
])

controls_row4 = widgets.HBox([
    districts_toggle,
    update_button
])

control_panel = widgets.VBox([
    widgets.HTML("<h3>🌡️ ERA5-Land Heat Metrics Map Explorer</h3>"),
    widgets.HTML("<p><b>Explore heat patterns using real ERA5-Land data - all metrics show day counts per year:</b></p>"),
    controls_row1,
    controls_row2,
    controls_row3,
    controls_row4,
    status_output
])

# Display the complete interface
display(control_panel)
display(m)

print("\n🎯 Heat Metrics Explorer Ready!")
print("\n📋 Instructions:")
print("1. Select a country from the dropdown")
print("2. Choose analysis year and baseline period")
print("3. Select the heat metric to visualize")
print("4. Adjust thresholds using the sliders (when applicable)")
print("5. Optionally toggle district boundaries")
print("6. Click 'Update Map' to generate visualization")
print("\n🌟 All Metrics Return Day Counts:")
print("• Fixed Threshold: Days per year above manual temperature threshold")
print("• Percentile Exceedance: Days per year above climate percentiles (90th, 95th, 99th)")
print("• EHF Heat Wave Days: Days meeting different EHF heat stress levels")
print("• Adjusted Threshold: Days above percentile + manual adjustment")
print("\n🔧 Features:")
print("• Real ERA5-Land temperature data (11km resolution)")
print("• geoBoundaries administrative boundaries")  
print("• Dynamic color scaling with heat-focused visualization")
print("• Interactive threshold controls")
print("• Standard climate baseline periods (1981-2010, 1961-1990)")

Map(center=[20, 0], controls=(WidgetControl(options=['position', 'transparent_bg'], position='topright', trans…


🎯 Heat Metrics Explorer Ready!

📋 Instructions:
1. Select a country from the dropdown
2. Choose analysis year and baseline period
3. Select the heat metric to visualize
4. Adjust thresholds using the sliders (when applicable)
5. Optionally toggle district boundaries
6. Click 'Update Map' to generate visualization

🌟 All Metrics Return Day Counts:
• Fixed Threshold: Days per year above manual temperature threshold
• Percentile Exceedance: Days per year above climate percentiles (90th, 95th, 99th)
• EHF Heat Wave Days: Days meeting different EHF heat stress levels
• Adjusted Threshold: Days above percentile + manual adjustment

🔧 Features:
• Real ERA5-Land temperature data (11km resolution)
• geoBoundaries administrative boundaries
• Dynamic color scaling with heat-focused visualization
• Interactive threshold controls
• Standard climate baseline periods (1981-2010, 1961-1990)
